this set up was made with the help of this guide https://nbviewer.org/github/microsoft/AIforEarthDataSets/blob/main/data/sentinel-5p.ipynb#Auth-files

# Environment setup

In [1]:
import os
import fsspec
import json
import urllib3
import xarray as xr
import numpy as np
from azure.storage.blob import ContainerClient
from datetime import datetime, timezone

# Not used directly, but needs to be installed to read NetCDF files with xarray
import h5netcdf

# get Product & Date

In [2]:
input_date = input('enter date [yyyy/mm/dd]: ')
input_gas = input('enter your choice of Gas (O3 / NO2 / CH4): ')

if input_gas == 'CH4':
    product = 'L2__CH4___'
else:
    if input_gas == 'NO2':
        product = 'L2__NO2___'
    else:
        product = 'L2__O3____'
        
date = input_date

enter date [yyyy/mm/dd]: 2022/03/01
enter your choice of Gas (O3 / NO2 / CH4): CH4


# get Token from URL

In [3]:
def get_Token_Data():
    http = urllib3.PoolManager()
    response = http.request('GET', 'https://planetarycomputer.microsoft.com/api/sas/v1/token/sentinel5euwest/sentinel-5p')
    data = json.loads(response.data.decode('utf-8'))
    return data

# Check Token File

## helper functions

In [4]:
def is_file_empty(file_path):
    # check if file exist and is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0

def getTime():
    dt = datetime().now().astimezone(timezone.pst)
    dt_string = dt.isoformat(timespec = 'milliseconds').replace('+00:00', 'Z')
    return dt_string
    
def checkToken(jsonData):
    token_expiration = jsonData['msft:expiry']
    currTime = getTime()
    if token_expiration <= currTime :
        print('token expired! expiration Time: ', currTime)
        return 0
    else:
        print('token still Valid! expiration Time ', curTime)
        return 1

## Read Token

In [5]:
sas_path = './tokens/sentinel-5p_sas.json'
is_empty = is_file_empty(sas_path)

if is_empty != 0:
    with open(sas_path, 'r+') as f:
        data = json.load(f)
        if checkToken(data):
            sas_token=data['token']
        else:
            newData = get_Token_Data()
            sas_token = newData['token']
            f.seek(0)
            json.dump(newData, f)
else:
    with open(sas_path, 'r+') as f:
        newData = get_Token_Data()
        sas_token = newData['token']
        f.seek(0)
        json.dump(newData, f)

# Azure storage constants

In [6]:
storage_account_name = 'sentinel5euwest'
container_name = 'sentinel-5p'
storage_account_url = 'https://' + storage_account_name + '.blob.core.windows.net/'

container_client = ContainerClient(account_url=storage_account_url, 
                                                container_name=container_name,
                                                credential=sas_token)

# List products matching our product/date

In [7]:
prefix = '/'.join(['TROPOMI',product,date])
print('Searching for prefix {}'.format(prefix))
generator = container_client.list_blobs(name_starts_with=prefix)
scene_paths = [blob.name for blob in generator]
print('\nFound {} matching scenes:\n'.format(len(scene_paths)))
for s in scene_paths:
    print(s.split('/')[-1])

Searching for prefix TROPOMI/L2__CH4___/2022/03/01

Found 14 matching scenes:

S5P_OFFL_L2__CH4____20220301T004505_20220301T022635_22695_02_020301_20220302T164456.nc
S5P_OFFL_L2__CH4____20220301T022635_20220301T040805_22696_02_020301_20220302T181137.nc
S5P_OFFL_L2__CH4____20220301T040805_20220301T054936_22697_02_020301_20220302T200810.nc
S5P_OFFL_L2__CH4____20220301T054936_20220301T073106_22698_02_020301_20220302T220046.nc
S5P_OFFL_L2__CH4____20220301T073106_20220301T091237_22699_02_020301_20220302T234452.nc
S5P_OFFL_L2__CH4____20220301T091237_20220301T105407_22700_02_020301_20220303T012017.nc
S5P_OFFL_L2__CH4____20220301T105407_20220301T123537_22701_02_020301_20220303T031500.nc
S5P_OFFL_L2__CH4____20220301T123537_20220301T141708_22702_02_020301_20220303T044722.nc
S5P_OFFL_L2__CH4____20220301T141708_20220301T155838_22703_02_020301_20220303T061839.nc
S5P_OFFL_L2__CH4____20220301T155838_20220301T174008_22704_02_020301_20220303T075219.nc
S5P_OFFL_L2__CH4____20220301T174008_20220301T192139

# Print Metadata for one scene

In [8]:
#different gas use different sc_mode, default CH4
sc_mode = 'OFFL'
if (product == 'L2__NO2___'):
    sc_mode = 'NRTI'
#need to add sc_mode for the other gases. will probably move this part to the beginning

offl_scenes = [s for s in scene_paths if sc_mode in s]
scene_path = offl_scenes[len(offl_scenes) // 2]
url = storage_account_url + container_name + '/' + scene_path
print('Processing image at URL:\n{}'.format(url))

Processing image at URL:
https://sentinel5euwest.blob.core.windows.net/sentinel-5p/TROPOMI/L2__CH4___/2022/03/01/S5P_OFFL_L2__CH4____20220301T123537_20220301T141708_22702_02_020301_20220303T044722/S5P_OFFL_L2__CH4____20220301T123537_20220301T141708_22702_02_020301_20220303T044722.nc


## Sign URL

In [9]:
import planetary_computer
import pystac
import rasterio

#item: pystac.Item = ... # no idea what goes here been trying multiple things but still stuck
b4_href = planetary_computer.sign(url) #i think this sign the url directly

print(b4_href)
#this print a url if clicked would download the scene




https://sentinel5euwest.blob.core.windows.net/sentinel-5p/TROPOMI/L2__CH4___/2022/03/01/S5P_OFFL_L2__CH4____20220301T123537_20220301T141708_22702_02_020301_20220303T044722/S5P_OFFL_L2__CH4____20220301T123537_20220301T141708_22702_02_020301_20220303T044722.nc?st=2022-04-30T18%3A07%3A57Z&se=2022-05-01T18%3A52%3A57Z&sp=rl&sv=2020-06-12&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2022-04-30T01%3A22%3A08Z&ske=2022-05-07T01%3A22%3A08Z&sks=b&skv=2020-06-12&sig=E8ydAgNeuwk%2BHwyZ8JZ3uyG%2B8MeyigUVQiBOboPUMIs%3D


In [10]:
def _create_dirs(dest_path):
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    elif not os.path.isdir(dest_path):
        shutil.rmtree(dst_path)
        os.makedirs(dest_path)

# We cannot access and read the files directly from the blob. We will need to download it first.
blob_list = container_client.list_blobs(name_starts_with=prefix)
for blob in blob_list:
        fname = os.path.join('data', blob.name)
        print(f'Downloading {blob.name} to {fname}')

        # get blob client which has download_blob method
        blob_client = container_client.get_blob_client(blob)
        _create_dirs(os.path.dirname(fname))
        with open(fname, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

In [11]:
data = 'data\TROPOMI/L2__CH4___/2022/03/01/S5P_OFFL_L2__CH4____20220301T004505_20220301T022635_22695_02_020301_20220302T164456/S5P_OFFL_L2__CH4____20220301T004505_20220301T022635_22695_02_020301_20220302T164456.nc'

In [12]:
from netCDF4 import Dataset

fh = Dataset(data, mode = 'r')

print(fh)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: KNMI/SRON
    source: Sentinel 5 precursor, TROPOMI, space-borne remote sensing, L2
    history: 2022-03-02 19:17:53 f_s5pops tropnll2dp /mnt/data1/storage_offl_l2/cache_offl_l2/WORKING-610672580/JobOrder.610672551.xml
    summary: TROPOMI/S5P Methane 1-Orbit L2 Swath 5.5x7.0km
    tracking_id: 5acbcf38-5b5a-4ce4-9922-786c6b8de93a
    id: S5P_OFFL_L2__CH4____20220301T004505_20220301T022635_22695_02_020301_20220302T164456
    time_reference: 2022-03-01T00:00:00Z
    time_reference_days_since_1950: 26357
    time_reference_julian_day: 2459639.5
    time_reference_seconds_since_1970: 1646092800
    time_coverage_start: 2022-03-01T01:06:40Z
    time_coverage_end: 2022-03-01T02:05:03Z
    time_coverage_duration: PT3503.561S
    time_coverage_resolution: PT0.840S
    orbit: 22695
    references: http://www.tropomi.eu/data-products/methane https://sentinels.copernicus

In [13]:
ds = xr.open_dataset(data)
print(ds)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/53)
    Conventions:                        CF-1.7
    institution:                        KNMI/SRON
    source:                             Sentinel 5 precursor, TROPOMI, space-...
    history:                            2022-03-02 19:17:53 f_s5pops tropnll2...
    summary:                            TROPOMI/S5P Methane 1-Orbit L2 Swath ...
    tracking_id:                        5acbcf38-5b5a-4ce4-9922-786c6b8de93a
    ...                                 ...
    product_version:                    1.4.0
    processing_status:                  Nominal
    Status_MET_2D:                      Nominal
    Status_CTM_CO:                      Nominal
    Status_CTMCH4:                      Nominal
    Status_NPP_VIIRS:                   Nominal


In [14]:
with fsspec.open(data) as f:
    ds = xr.open_dataset(f,group='/PRODUCT')
print(ds)

<xarray.Dataset>
Dimensions:                              (scanline: 4172, ground_pixel: 215, time: 1, corner: 4, layer: 12, level: 13)
Coordinates:
  * scanline                             (scanline) float64 0.0 ... 4.171e+03
  * ground_pixel                         (ground_pixel) float64 0.0 ... 214.0
  * time                                 (time) datetime64[ns] 2022-03-01
  * corner                               (corner) float64 0.0 1.0 2.0 3.0
  * layer                                (layer) float64 0.0 1.0 ... 10.0 11.0
  * level                                (level) float64 0.0 1.0 ... 11.0 12.0
    latitude                             (time, scanline, ground_pixel) float32 ...
    longitude                            (time, scanline, ground_pixel) float32 ...
Data variables:
    delta_time                           (time, scanline) datetime64[ns] ...
    time_utc                             (time, scanline) object ...
    qa_value                             (time, scanline, g